# Лаба 8. Рекомендательная система фильмов MovieLens 

## User-User Recommender System 

In [ ]:
import pandas as pd
import numpy as np
from math import sqrt
import json
import pandas as pd
import numpy as np
from math import sqrt
import json
import sys


## Константы из ЛК

In [45]:
id_to_predict = 524

## Загрузим датасеты

`u.data` содержит все оценки

In [5]:
df_data = pd.read_csv(
    './ml-100k/u.data', 
    delimiter='\t', 
    header=None, 
    names=['user_id', 'item_id','rating','timestamp']
)

In [6]:
df_data

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


`u.item` — список всех фильмов

In [9]:
item_col_lst = [
    
    'movie_id',
    'movie_title',
    'release_date',
    'video_release_date',
    'imdb_url',
    'unknown',
    'action',
    'adventure',
    'animation',
    'children',
    'comedy',
    'crime',
    'documentary',
    'drama',
    'fantasy',
    'film-Noir', 
    'horror',
    'musical',
    'mystery',
    'romance',
    'sci-Fi',
    'thriller',
    'war',
    'western',

]

In [36]:
sys.stdin = open('./ml-100k/u.item', 'r')
df_item  = []

for ext_line, line in enumerate(sys.stdin):
    try:
        part1, part2 = line.strip().split('||')
        item, name, item_date = part1.strip().split('|')    
        df_item.append([item, name, item_date])
    except:
        pass
    
df_item = pd.DataFrame(df_item, columns=['movie_id', 'name', 'item_date'])  
df_item['movie_id'] = df_item['movie_id'].astype('int')

In [ ]:
df_item = pd.read_csv(
    './ml-100k/u.item', 
    delimiter='|', 
    header=None, 
    names=item_col_lst,
    index_col=None
)

In [6]:
df_item

,movie_id,movie_title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,children,...,fantasy,film-Noir,horror,musical,mystery,romance,sci-Fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
print(df_item.shape)
df_item.head()

(1681, 3)


,itemid,name,item_date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995


In [33]:
df_item.dtypes

itemid        int32
name         object
item_date    object
dtype: object

In [14]:
from tqdm import tqdm_notebook

In [41]:
pd.DataFrame(group.head(10)).dot(pd.DataFrame(group.head(10)).T)

item_id,1,2,3,4,5,6,7,8,9,10
item_id,,,,,,,,,,
1,3073009,736260,478569,1300726,497852,163029,2610217,1533875,2042245,597773
2,736260,176400,114660,311640,119280,39060,625380,367500,489300,143220
3,478569,114660,74529,202566,77532,25389,406497,238875,318045,93093
4,1300726,311640,202566,550564,210728,69006,1104838,649250,864430,253022
5,497852,119280,77532,210728,80656,26412,422876,248500,330860,96844
6,163029,39060,25389,69006,26412,8649,138477,81375,108345,31713
7,2610217,625380,406497,1104838,422876,138477,2217121,1302875,1734685,507749
8,1533875,367500,238875,649250,248500,81375,1302875,765625,1019375,298375
9,2042245,489300,318045,864430,330860,108345,1734685,1019375,1357225,397265


In [10]:
len(lst)

2829124

## Основные характеристики данных MovieLens

### Найдите количество всех пользователей и количество всех фильмов в данных (общее число пользователей и фильмов в датасете).

In [164]:
user_count = len(df_data.user_id.unique())
movie_count = len(df_item.movie_id.unique())

print(user_count, movie_count)

943 1682


### Сколько пользователь в среднем ставит рейтингов? Подсчитать `количество рейтингов / количество пользователей`.

In [38]:
average_user_ratings = len(df_data) / user_count
average_user_ratings

106.04453870625663

### Сколько фильм в среднем имеет рейтингов? Подсчитать`количество рейтингов / количество фильмов`.

In [39]:
average_film_ratings = len(df_data) / movie_count
average_film_ratings

59.4883997620464

### Найдите процент заполненных ячеек в данных: `количество рейтингов / (количество пользователей * количество фильмов)`.

In [40]:
completeness = len(df_data) / (user_count*movie_count)
completeness

0.06308419911139597

## User-user CF

### Для каждого пользователя найдите его средний рейтинг (сумма рейтингов пользователя/количество рейтингов пользователя).

In [41]:
r = df_data.groupby('user_id')['rating'].mean()

In [43]:
r

user_id
1      3.610294
2      3.709677
3      2.796296
4      4.333333
5      2.874286
         ...   
939    4.265306
940    3.457944
941    4.045455
942    4.265823
943    3.410714
Name: rating, Length: 943, dtype: float64

In [42]:
target_user_avg = r[id_to_predict]

### Посчитайте меру близости Пирсона выданного вам пользователя со всеми остальными пользователями.

In [47]:
# Какие фильмы оценил целевой пользователь?
target_user_movies = set(
    df_data[df_data.user_id == id_to_predict].item_id
)

In [48]:
# сколько всего фильмов он оценил?
len(target_user_movies)

306

In [49]:
corr_lst = []

# Переберем всех пользователей в цикле
for user in df_data.user_id.unique():
    # Выделим фильмы оцененные пользователем
    user_movies = set(
        df_data[df_data.user_id == user].item_id
    )
    
    # Найдем пересечения в фильмах с целевым пользоватлем
    intersect = target_user_movies & user_movies
    if len(intersect) == 0: continue
    
   
    numerator = 0
    denominator1 = 0
    denominator2 = 0
    
    for movie in intersect:
        # Посчитаем числитель
        ra = df_data[
                (df_data.item_id == movie) & 
                (df_data.user_id == user)
            ]['rating'].values[0]
        
        ra_avg = r[user]
        
        ru = df_data[
                (df_data.item_id == movie) & 
                (df_data.user_id == id_to_predict)
            ]['rating'].values[0]
        
        ru_avg = target_user_avg
        
        numerator += (ra - ra_avg)*(ru - ru_avg)
        
        # Посчитаем знаменатель
        denominator1 += (ra - ra_avg)**2
        denominator2 += (ru - ru_avg)**2
        
    # Поправочный коэффициент на нехватку данных
    coef = min((len(intersect)/50),1)
    
    # Рассчитаем корреляцию
    s = (coef * numerator) / (sqrt(denominator1)*sqrt(denominator2))
        
        
    corr_lst.append([user, s])

In [50]:
df_s = pd.DataFrame(corr_lst, columns=['user_id','correlation'])

In [51]:
df_s

,user_id,correlation
0,196,-0.022784
1,186,-0.031638
2,22,0.072344
3,244,0.020513
4,166,-0.006023
...,...,...
938,939,0.061249
939,936,0.295666
940,930,0.164994
941,920,-0.021979


### Найдите 30 ближайших пользователей-соседей данного пользователя

In [52]:
pearson_neighbours = df_s.sort_values(
    by='correlation', 
    ascending=False).iloc[1:31,].user_id.tolist()

### Для всех фильмов найдем прогноз оценки

In [53]:
df_neighbours = df_s[df_s.user_id.isin(pearson_neighbours)]\
                .merge(df_data, how="left", on='user_id')
df_neighbours

,user_id,correlation,item_id,rating,timestamp
0,115,0.437505,265,2,881171488
1,115,0.437505,20,3,881171009
2,115,0.437505,1067,4,881171009
3,115,0.437505,8,5,881171982
4,115,0.437505,127,5,881171760
...,...,...,...,...,...
5128,894,0.458981,16,3,880993614
5129,894,0.458981,330,3,880415951
5130,894,0.458981,166,4,882404306
5131,894,0.458981,750,4,883518875


In [222]:
recom_lst = []

# Переберем все фильмы
for movie in df_item.movie_id.unique():
    
    # Проверим смотрели ли соседи данный фильм
    df_movie_exist = df_neighbours[df_neighbours.item_id == movie]
    if len(df_movie_exist) == 0: continue
        
        
    numerator = 0
    denominator = 0
    
    for user in df_movie_exist.user_id:
        # Считаем числитель
        # Мера близости пользователя
        s = df_movie_exist[df_movie_exist.user_id == user]['correlation'].values[0]
        
        ru = df_movie_exist[
                (df_movie_exist.item_id == movie) & 
                (df_movie_exist.user_id == user)]['rating'].values[0]
        
        ru_avg = r[user]
        
        numerator += s*(ru - ru_avg)
        
        # Считаем знаменатель
        denominator += abs(s)
    
    # Средний рейтинг целевого пользователя          
    ra = r[id_to_predict]
    
    # Итоговая рекомендация
    recom = ra + numerator / denominator
    
    recom_lst.append([movie, recom])

In [223]:
df_recom = pd.DataFrame(recom_lst, columns=['item_id','rating'])
df_recom

,item_id,rating
0,1,3.562200
1,2,2.776399
2,3,2.746245
3,4,3.605320
4,5,3.049536
...,...,...
1017,1598,5.271242
1018,1609,3.271242
1019,1610,2.271242
1020,1611,3.271242


### Рекомендуйте пользователю 10 фильмов (`pearson_top10`), которые он еще не оценивал, с самыми высокими прогнозами из предыдущего пункта.

In [224]:
# Вычислим что пользователь ещё не смотрел
inverse_boolean_series = ~df_recom.item_id.isin(list(target_user_movies))
df_not_watched = df_recom[inverse_boolean_series]


pearson_top10 = df_not_watched.sort_values(
                    by=['rating','item_id'], 
                     ascending=(False, True)).head(10)['item_id'].tolist()

## Результат

In [228]:
pearson_top10

[1137, 1598, 1128, 1368, 487, 536, 863, 1211, 251, 1266]

In [229]:
result_dic = {
    "average_film_ratings": round(average_film_ratings, 4),
    "average_user_ratings": round(average_user_ratings, 4),
    "completeness": round(completeness, 4),
    "pearson_neighbours": pearson_neighbours,
    "pearson_top10": pearson_top10
}

In [230]:
with open('../lab08.json', 'w') as outfile:
    json.dump(result_dic, outfile)